In [157]:
import pandas as pd
import numpy as np
duda=pd.read_excel('Duda_MSSNG.xlsx',sheet_name="Table S4", skiprows=1)
novel=duda.dropna(subset=['OMIM_PHENOTYPE'])

In [158]:
novel = duda[duda['OMIM_ID'].isna()]
novel_list = list(set(novel["GENE_SYMBOL"].tolist()))

mssng_sig=duda.dropna(subset=['OMIM_PHENOTYPE'])
mssng_sig_list = list(set(mssng_sig["GENE_SYMBOL"].tolist()))

lof=list(set(duda["GENE_ID"].tolist()))

complete_list = duda["GENE_ID"].tolist()

len(lof)

214

In [159]:
def gmt_creator(gene_lists, name):


    from gprofiler import GProfiler
    gmt_strigs = []
    
    for gene_list, name in zip(gene_lists, name):
        gp = GProfiler(return_dataframe=True)
        gp_convert_df =gp.convert(organism  = 'hsapiens',
                                    query= gene_list,
                                        target_namespace= 'ENSG',
                                        numeric_namespace='ENTREZGENE_ACC',)
        gp_convert_df = gp_convert_df[['incoming', 'converted']]
        ensemble_list = gp_convert_df['converted'].tolist()
        gene_string = '\t'.join(ensemble_list)
        gmt_line= f'{name}\tMutation: {name}\t' + gene_string
        gmt_strigs.append(gmt_line)
        print(gmt_line)

    with open('mssng.gmt', 'w') as f:
        for gmt_line in gmt_strigs:
            f.write(gmt_line + "\n")     


In [160]:
gmt_creator([novel_list, mssng_sig_list, lof,complete_list], ['MSSNG_NOVEL', 'MSSNG_SIG','MSSNG_LOF',"COMPLETE"])

MSSNG_NOVEL	Mutation: MSSNG_NOVEL	None	ENSG00000137073	ENSG00000169718	ENSG00000166164	ENSG00000144504	ENSG00000119285	ENSG00000163625	ENSG00000149532	ENSG00000119321	ENSG00000142235	ENSG00000181458	ENSG00000084710	ENSG00000164151	ENSG00000131969	ENSG00000101331	ENSG00000196865	ENSG00000176273	ENSG00000174946	ENSG00000186364	ENSG00000166348	ENSG00000138757	None	ENSG00000198885	ENSG00000183530	ENSG00000182898	ENSG00000163214	ENSG00000239388	ENSG00000197863	ENSG00000176994	ENSG00000109118	ENSG00000221909	ENSG00000261787	ENSG00000160058	ENSG00000173064	ENSG00000172199	None	ENSG00000127080	ENSG00000197748	ENSG00000198870
MSSNG_SIG	Mutation: MSSNG_SIG	ENSG00000075043	ENSG00000095002	ENSG00000126524	ENSG00000140199	ENSG00000171316	ENSG00000100288	ENSG00000023287	ENSG00000132326	ENSG00000101126	ENSG00000011201	ENSG00000171862	ENSG00000115657	None	ENSG00000144285	ENSG00000087470	ENSG00000169057	ENSG00000148795	ENSG00000148606	ENSG00000112039	ENSG00000106804	ENSG00000115919	ENSG00000053747	ENSG

# Test Gprofiler
 



        
          
   
    
     
      
       
        
         
          

In [ ]:
results=pd.read_csv('svc_clean.csv')
gene_list=results['Ensembl_ID'].tolist()
len(gene_list)


data = pd.read_csv('svc_clean.csv')# chage data as need be

data["Decile_rank"] = pd.qcut(data["Probability_Class_1"], 10, labels=False)
data["Decile_rank"] = data["Decile_rank"].astype('category')
data["Decile_rank"] = data["Decile_rank"].cat.rename_categories([f'D{i}' for i in range(10,0,-1)])

deciles = {}
for i in range(1, 11):
        deciles[f'd_{i}'] = data[data["Decile_rank"] == f'D{i}'].copy()



d_1=deciles['d_1']['Ensembl_ID'].tolist()
len(d_1)

In [84]:
import requests
import pandas as pd
import numpy as np
from gprofiler import GProfiler

def get_token_form_response(response):
    if response.status_code == 200:
        token = response.json()['organism']
    else:
        try:
            error_message = 'Error: {}'.format(response.json()['message'])
        except:
            error_message = 'Error, status code {}'.format(response.status_code)
        raise AssertionError(error_message)
    #print("Token:", token)
    return token

def main_custom(deciles, gmt_file,mutation_name):


    dataframes= []
    for i,key in enumerate(deciles):
        decile_list = deciles[key]["Ensembl_ID"].to_list()
        print(f'Processing {key,mutation_name}...')

        gp = GProfiler(
            user_agent='gprofiler_custom_gmt', #optional user agent
            return_dataframe=True, #return pandas dataframe or plain python structures    
        )

        with open(gmt_file) as f:
            response = requests.post('https://biit.cs.ut.ee/gprofiler/api/gost/custom/',
                            json={'gmt':f.read(),
                                'name': gmt_file})
        token = get_token_form_response(response)

        decile_results = gp.profile(query=decile_list, organism=token,user_threshold=1,domain_scope="custom",background=d_1)
        decile_results.insert(0, "Decile", i+1)
        decile_results=decile_results[decile_results['native']==mutation_name]
        dataframes.append(decile_results)


    data = pd.concat(dataframes)
    data= data.sort_values(by='Decile', ascending=False).reset_index(drop=True)
    data['Intersection_to_Term_Ratio'] = data['intersection_size'] / data['term_size']
    data=data.sort_values(by='Decile', ascending=True).reset_index(drop=True)

    return data


In [85]:
MSSNG_lof = main_custom(deciles, 'krish_duda.gmt','prb_rec_LGD'),"MSSNG_lof" # native is the first 
MSSNG_lof[0]

Processing ('d_1', 'prb_rec_LGD')...
Processing ('d_2', 'prb_rec_LGD')...
Processing ('d_3', 'prb_rec_LGD')...
Processing ('d_4', 'prb_rec_LGD')...
Processing ('d_5', 'prb_rec_LGD')...
Processing ('d_6', 'prb_rec_LGD')...
Processing ('d_7', 'prb_rec_LGD')...
Processing ('d_8', 'prb_rec_LGD')...
Processing ('d_9', 'prb_rec_LGD')...
Processing ('d_10', 'prb_rec_LGD')...


C:\Users\joaop\AppData\Local\Temp\ipykernel_13580\1555620758.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = pd.concat(dataframes)


,Decile,source,native,name,p_value,significant,description,term_size,query_size,intersection_size,effective_domain_size,precision,recall,query,parents,Intersection_to_Term_Ratio
0,1,krish_duda,prb_rec_LGD,Mutation: prb_rec_LGD,1.0,False,Mutation: prb_rec_LGD,20,291,20,291,0.068729,1.0,query_1,[],1.0
